# Semi-structured RAG

## Load environment variables

In [1]:
from dotenv import load_dotenv

load_dotenv('app/src/shared/.env')

True

## Load data 

As normas brasileiras relacionadas a instalações elétricas abrangem uma variedade de aspectos, desde baixa tensão até instalações específicas em locais como áreas classificadas e sistemas fotovoltaicos. As principais normas da Associação Brasileira de Normas Técnicas (ABNT) sobre instalações elétricas incluem:

* NBR 5410 - Instalações elétricas de baixa tensão: Trata das condições para o projeto, execução e manutenção de instalações elétricas de baixa tensão.

* NBR 14039 - Instalações elétricas de média tensão de 1,0 kV a 36,2 kV: Estabelece as condições para projeto, execução e manutenção dessas instalações.

* NBR 5413 - Iluminância de interiores: Define os requisitos para níveis de iluminância em ambientes internos.

* NBR 13570 -  Instalações Elétricas em Locais de Afluência de Público - Requisitos específicos

* ABNT NBR IEC 60079-14 - Instalações elétricas em atmosferas explosivas - Área classificada: Requisitos para instalações em áreas com risco de explosão.

* NBR 10898 - Sistemas de iluminação de emergência: Especifica os requisitos para sistemas de iluminação de emergência em edifícios.

* NBR 15514 - Recipientes transportáveis de gás liquefeito de petróleo (GLP) — Área de armazenamento — Requisitos de segurança
    * Embora a norma NBR 15514 não seja diretamente uma norma de instalações elétricas, ela possui interseções com a área elétrica em aspectos relacionados à segurança, especialmente devido aos riscos potenciais de explosões e incêndios associados ao GLP.
* NBR 5419 - Proteção contra descargas atmosféricas (em quatro partes):

    * Parte 1: Princípios gerais
    * Parte 2: Gerenciamento de risco
    * Parte 3: Danos físicos a edificações e perigos à vida
    * Parte 4: Sistemas elétricos e eletrônicos internos na estrutura

    * NBR 16280 - Reforma em edificações - Sistema de gestão de reformas: Estabelece requisitos para reformas em edificações, incluindo instalações elétricas.


Essas são algumas das principais normas que tratam de diversos aspectos das instalações elétricas no Brasil, garantindo segurança, eficiência e conformidade técnica.

In [ ]:
import os
import logging
from concurrent.futures import ThreadPoolExecutor
from typing import List
from unstructured.partition.pdf import partition_pdf

# Configure logging 
logging.basicConfig(filename='pdf_processing.log', level=logging.INFO, format='%(asctime)s %(message)s')

def process_pdf_file(filename, path):
    """Processes a single PDF file and returns the extracted elements.

    Logs information and errors during processing.
    """
    try:
        raw_pdf_elements = partition_pdf(
            filename=os.path.join(path, filename),  # Combine path and filename
            extract_images_in_pdf=False,
            infer_table_structure=True,
            chunking_strategy="by_title",
            max_characters=4000,
            new_after_n_chars=3800,
            combine_text_under_n_chars=2000
        )
        logging.info(f"Successfully processed PDF: {filename}")
        return raw_pdf_elements
    except Exception as e:  # Catch specific PDF processing errors
        logging.error(f"Error processing PDF: {filename} - {e}")
        return []

def process_single_pdf(pdf_folder_path: str, filename: str):
    """Helper function to process a single PDF file."""
    if filename.endswith(".pdf"):
        logging.info(f"Reading PDF doc: {filename}")
        return process_pdf_file(filename, pdf_folder_path)
    return []

# Loop through PDF Files with ThreadPoolExecutor:
def process_multiple_pdfs(pdf_folder_path: str, max_workers: int = 8) -> List:
    """Processes all PDF files within a specified folder using ThreadPoolExecutor.

    Logs information and errors during processing.
    """
    all_elements = []
    filenames = [filename for filename in os.listdir(pdf_folder_path) if filename.endswith(".pdf")]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = executor.map(lambda filename: process_single_pdf(pdf_folder_path, filename), filenames)

    for result in results:
        all_elements.extend(result)  # Extend the all_elements list

    return all_elements

# pdf_folder_path = "/Users/nathan/workspace/tcc/app/src/database/pdf/"
# raw_pdfs_elements = process_multiple_pdfs(pdf_folder_path, max_workers=10)

In [ ]:
pdf_folder_path = "/Users/nathan/workspace/tcc/test"
raw_pdfs_elements = process_multiple_pdfs(pdf_folder_path, max_workers=4)

In [ ]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdfs_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

In [ ]:
from typing import Any
from pydantic import BaseModel

# Define the Element class based on potential types returned by partition_pdf
class Element(BaseModel):
    type: str  # Textual representation of the element type (e.g., "table", "text")
    text: Any  # Content of the element, can be text, tables, or other data structures

# Categorize by type
def categorize_elements(raw_pdf_elements) -> list[Element]:
    """Categorizes elements by type and returns a dictionary with counts."""
    categorized_elements = [] # Initialize category counts
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            categorized_elements.append(Element(type="table", text=str(element)))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            categorized_elements.append(Element(type="text", text=str(element)))
    return categorized_elements

In [ ]:
# Improved categorization function with clear structure
all_categories = categorize_elements(raw_pdfs_elements)
print(len(all_categories))

# Tables
table_elements = [e for e in all_categories if e.type == "table"]
print(f"Number of tables: {len(table_elements)}")

# Text
text_elements = [e for e in all_categories if e.type == "text"]
print(f"Number of text elements: {len(text_elements)}")

### Calculate the number of tokens for each document

In [ ]:
text_elements

### Cleaning the text elements

In [ ]:
import re

def cleaning_text(text_element: str)-> str:
    
    # Remover quebras de linha desnecessárias
    text = re.sub(r'\n+', ' ', text_element)
    
    # Remover números de página e sequências de dígitos isolados
    text = re.sub(r'\s\d+\s', ' ', text)
    
    # Remover espaços extras
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Corrigir espaços antes de pontuação
    text = re.sub(r'\s+([,.;:])', r'\1', text)
    
    return text

In [ ]:
# Apply to texts
texts = [i.text for i in text_elements]

cleaned_texts = [cleaning_text(text) for text in texts]

In [ ]:
# Exibir os textos limpos
for cleaned_text in cleaned_texts:
    print(cleaned_text)

In [ ]:
texts

## Summarizing tables and text chunks from pdfs

In [ ]:
from langchain_google_genai import GoogleGenerativeAI

model = GoogleGenerativeAI(model="gemini-pro",
                 temperature=0.3, top_p=0.85)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Prompt
prompt_text = """Você é um assistente com a tarefa de resumir tabelas e textos de normas brasileiras sobre instalações elétricas.
Faça um resumo sucinto da tabela ou do texto a seguir. Tabela ou texto: {element} """

prompt = ChatPromptTemplate.from_template(prompt_text)

print(prompt)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [ ]:
# Apply to tables
tables = [i.text for i in table_elements]
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 4})

In [ ]:
# Apply to texts
# texts = [i.text for i in text_elements]
text_summaries = summarize_chain.batch(cleaned_texts, {"max_concurrency": 4})

In [ ]:
print(len(texts))
print(len(cleaned_texts))
print(len(text_summaries))

### Data analytics

In [ ]:
# analisar os crunks pai com filho(summarized by gemini model)
import pandas as pd

text_data = {"Texts": texts, 
             "Cleaned Texts": cleaned_texts,
             "Text_summaries": text_summaries}

df = pd.DataFrame(text_data)

df.head()

# # Configurar a largura máxima da coluna
# pd.set_option('display.max_colwidth', None)

# # Exibir as 5 primeiras linhas do DataFrame
# print(df.head(9))

In [ ]:
df['Text_summaries'].iloc[12]

## Add to vectorstore
Use Multi Vector Retriever with summaries:

- InMemoryStore stores the raw text, tables
- vectorstore stores the embedded summaries

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", task_type="retrieval_document")
gemini_embeddings

GoogleGenerativeAIEmbeddings(model='models/embedding-001', task_type='retrieval_document', google_api_key=None, credentials=None, client_options=None, transport=None)

In [2]:
import sqlite3
import pandas as pd

# Conexão ao banco de dados SQLite
conn = sqlite3.connect('chroma_db/test_summaries/chroma.sqlite3')


# Executa a consulta e retorna um DataFrame
df = pd.read_sql_query("SELECT * FROM collections", conn)

# Exibe o DataFrame
print(df,"\n\n")

conn.close()

                                     id            name  dimension  \
0  ee27171e-9b08-41c4-90d4-ebf3585f9347  summaries_nr10        768   

                            database_id  
0  00000000-0000-0000-0000-000000000000   




#### Creating the empty vector DB 

In [6]:
import sqlite3
import pandas as pd
from langchain_community.vectorstores import Chroma

persist_directory = "./chroma_db/test_summaries"

# inicianlizing the vectorstore
vectorstore = Chroma(collection_name="summaries_nr10", 
                        embedding_function=gemini_embeddings,
                        persist_directory=persist_directory)

# Conexão ao banco de dados SQLite
conn = sqlite3.connect('chroma_db/test_summaries/chroma.sqlite3')

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
print(tables, "\n")

# Executa a consulta e retorna um DataFrame
df = pd.read_sql_query("SELECT * FROM embedding_fulltext_search", conn)

# Exibe o DataFrame
print(df,"\n\n")

conn.close()

                                 name
0                          migrations
1                    embeddings_queue
2                 collection_metadata
3                            segments
4                    segment_metadata
5                             tenants
6                           databases
7                         collections
8                          embeddings
9                  embedding_metadata
10                         max_seq_id
11          embedding_fulltext_search
12     embedding_fulltext_search_data
13      embedding_fulltext_search_idx
14  embedding_fulltext_search_content
15  embedding_fulltext_search_docsize
16   embedding_fulltext_search_config 

                                         string_value
0   **Resumo da NR-10: Segurança em Instalações e ...
1   A NR 10 estabelece requisitos de segurança par...
2   **Medidas de Controle em Instalações Elétricas...
3   **Proteção Individual e Segurança em Projetos*...
4   **Resumo da Tabela/Texto sobre Segurança

#### Multi Vector Retriver

In [7]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

#### Add the documents to the vectorial database

In [ ]:
import uuid
from langchain_core.documents import Document

# Apply to tables
tables = [i.text for i in table_elements]
# Apply to texts
texts = [i.text for i in text_elements]

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
page_content_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]

vectorstore.add_documents(page_content_texts)
retriever.docstore.mset(list(zip(doc_ids, cleaned_texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
page_content_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]

vectorstore.add_documents(page_content_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))


In [ ]:
import sqlite3
import pandas as pd

# Conexão ao banco de dados SQLite
conn = sqlite3.connect('chroma_db/test_summaries/chroma.sqlite3')

# Executa a consulta e retorna um DataFrame
df = pd.read_sql_query("SELECT * FROM embedding_fulltext_search", conn)

# Exibe o DataFrame
print(df,"\n\n")
print(f"Colunas: {df.columns} \n")

# print(df['string_value'].iloc[1])

# Fecha a conexão
conn.close()

#### Visualization of the vector DB

In [ ]:
import sqlite3
import pandas as pd

# Conexão ao banco de dados SQLite
conn = sqlite3.connect('chroma_db/chroma.sqlite3')

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)

print(tables)

conn.close()

In [ ]:
import sqlite3
import pandas as pd

# Conexão ao banco de dados SQLite
conn = sqlite3.connect('chroma_db/chroma.sqlite3')

# Consulta SQL
query = "SELECT * FROM embedding_fulltext_search"

# Executa a consulta e retorna um DataFrame
df = pd.read_sql_query(query, conn)

# Exibe o DataFrame
print(df,"\n\n")
print(f"Colunas: {df.columns}")
print(df['string_value'].iloc[1])

# Fecha a conexão
conn.close()

## RAG Pipeline

### Retriver

#### Vector store-backed retriever

In [ ]:
# Load from disk
# vectorstore_db = Chroma(
#                         persist_directory="./chroma_db",       # Directory of db
#                         embedding_function=gemini_embeddings   # Embedding model
#                    )

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# retriever = vectorstore.as_retriever(
#     search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7}
# )

# retriever = vectorstore.as_retriever(search_type="mmr")

retriever

In [ ]:
query = "O que diz o topico 10.2.8.1 da norma regulamentadora NR 10?"
docs = retriever.invoke(query)
docs

#### Parent Document Retriever

#### Multivector Retriver

In [ ]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

persist_directory = "./chroma_db"

# Load from disk
vectorstore = Chroma(collection_name="summaries", 
                        embedding_function=gemini_embeddings,
                        persist_directory=persist_directory)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

### Generate

In [12]:
import textwrap
from IPython.display import Markdown

# def to_markdown(text):
#   text = text.replace('•', '  *')
#   return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def to_markdown(text):
    text = text.replace('•', '*')
    text = textwrap.dedent(text).strip()

    return Markdown(text)

#### With chat memory

In [15]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.retrieval import create_retrieval_chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap
from langchain_core.runnables import RunnablePassthrough



model = GoogleGenerativeAI(model="gemini-pro", temperature=0.3, top_p=0.85)


### Contextualize question ###
contextualize_system_prompt = """Dado um histórico de bate-papo e a última pergunta do usuário \
que pode fazer referência ao contexto no histórico de bate-papo, formule uma pergunta autônoma \
que possa ser compreendida sem o histórico do bate-papo. NÃO responda à pergunta, \
apenas reformule-a se necessário e, caso contrário, devolva-a como está."""


contextualize_system_prompt = ChatPromptTemplate.from_messages(
    [
        ("ai", contextualize_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(model, 
                                                         retriever, 
                                                         contextualize_system_prompt)


### Answer question ###
template = """Você é um assistente chamado Spark e sua função é responder dúvidas e questionamentos relacionadas as instalações elétricas brasileiras \
com base no contexto, o qual pode incluir textos e/ou tabelas referentes as normas brasileiras (NBRs): \
{context}
"""

prompt = ChatPromptTemplate.from_messages([("ai", template),
                                           MessagesPlaceholder("chat_history"),
                                           ("human", "{input}")])

question_answer_chain = create_stuff_documents_chain(model, prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Statefully manage chat history ###
store = {}
session_id = "default_session_id"

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
to_markdown(conversational_rag_chain.invoke(
    {"input": "O que diz a norma NR 10?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  
)["answer"])

In [ ]:
to_markdown(conversational_rag_chain.invoke(
    {"input": "Qual foi minha pergunta anterior?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  
)["answer"])

In [10]:
to_markdown(conversational_rag_chain.invoke(
    {"input": "Qual objetivo da norma regulamentadora NR 10?"},
    config={
        "configurable": {"session_id": session_id}
    },  
)["answer"])

Parent run 39dce1eb-b844-4818-b463-7c273db05583 not found for run 6df37174-9ff4-41e7-a1b8-be31decc8305. Treating as a root run.


> Estabelecer os requisitos e medidas de controle para garantir a segurança e a saúde dos trabalhadores que interagem com instalações e serviços em eletricidade.

In [11]:
to_markdown(conversational_rag_chain.invoke(
    {"input": "qual foi minha pergunta anterior?"},
    config={
        "configurable": {"session_id": session_id}
    },  
)["answer"])

Parent run fe2d2a71-31cd-4372-8906-1c61c910da6b not found for run 2831dfec-47b6-4194-895b-ca3e4b6a0b40. Treating as a root run.


> Qual objetivo da norma regulamentadora NR 10?

In [13]:
query = "O que diz o topico 10.2.8.1 da norma regulamentadora NR 10?" 

to_markdown(conversational_rag_chain.invoke(
    {"input":query},
    config={
        "configurable": {"session_id": session_id}
    },  
)["answer"])

Parent run 91367e4a-8e6d-4543-b3fe-6a174e46b018 not found for run dac12c2d-4ec6-4baa-be18-765db5037c5e. Treating as a root run.


Não tenho acesso ao conteúdo da norma regulamentadora NR 10, portanto, não posso fornecer informações sobre o tópico 10.2.8.1.

In [14]:
query = "Como assim voce n tem acesso ao conteudo da NR 10?" 

to_markdown(conversational_rag_chain.invoke(
    {"input":query},
    config={
        "configurable": {"session_id": session_id}
    },  
)["answer"])

Parent run a99d03e9-f0bd-4c99-b9e7-e93d50f675f3 not found for run b4cd587f-ddb0-4d1e-a7d4-aa299b87bec2. Treating as a root run.


Como um modelo de linguagem de IA, não tenho acesso direto ao conteúdo de documentos específicos, como a Norma Regulamentadora NR 10. Meu conhecimento é baseado em um vasto conjunto de dados de texto, que inclui informações sobre vários tópicos, mas não tenho a capacidade de acessar ou recuperar documentos específicos em tempo real.

In [16]:
query = "Me diga o que a NR 10 fala sobre EPIS (equipamentos de proteção individual)." 

to_markdown(conversational_rag_chain.invoke(
    {"input":query},
    config={
        "configurable": {"session_id": session_id}
    },  
)["answer"])

Parent run 915f542e-619c-4297-a138-6c669c3f7232 not found for run 7176e4a6-4d50-40a4-81b8-93a56588c32f. Treating as a root run.


**NR 10 - Segurança em Instalações e Serviços em Eletricidade**

**Capítulo 5 - Equipamentos de Proteção Individual (EPIs)**

**5.1 Disposições Gerais**

* Os EPIs devem ser adequados aos riscos existentes e às atividades a serem executadas.
* Os EPIs devem ser fornecidos gratuitamente pelo empregador.
* Os EPIs devem ser mantidos em bom estado de conservação e higienização.
* Os trabalhadores devem ser treinados para o uso correto dos EPIs.

**5.2 EPIs para Trabalhos em Instalações Elétricas**

**5.2.1 Luvas Isolantes**

* Devem ser utilizadas luvas isolantes de borracha natural ou sintética, com tensão de ensaio igual ou superior à tensão da instalação.
* Devem ser inspecionadas antes de cada uso e substituídas se apresentarem danos ou sinais de desgaste.

**5.2.2 Botas Isolantes**

* Devem ser utilizadas botas isolantes de borracha natural ou sintética, com tensão de ensaio igual ou superior à tensão da instalação.
* Devem ser inspecionadas antes de cada uso e substituídas se apresentarem danos ou sinais de desgaste.

**5.2.3 Capacete de Segurança**

* Deve ser utilizado capacete de segurança com aba frontal e viseira, conforme ABNT NBR 8221.

**5.2.4 Óculos de Segurança**

* Devem ser utilizados óculos de segurança com proteção lateral, conforme ABNT NBR 13694.

**5.2.5 Protetor Facial**

* Deve ser utilizado protetor facial quando houver risco de projeção de partículas ou líquidos.

**5.2.6 Cinto de Segurança**

* Deve ser utilizado cinto de segurança com talabarte e trava-quedas quando houver risco de queda.

**5.2.7 Roupas de Proteção**

* Devem ser utilizadas roupas de proteção confeccionadas em material antiestático e resistente ao fogo, conforme ABNT NBR 14725.

**5.2.8 Outros EPIs**

* Outros EPIs podem ser necessários dependendo dos riscos específicos da atividade, como:
    * Máscaras respiratórias
    * Protetores auriculares
    * Luvas de couro
    * Avental de borracha

#### Without chat memory 

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap


model = GoogleGenerativeAI(model="gemini-pro", temperature=0.3, top_p=0.85)

# Prompt template
template = """Você é um assistente chamado Spark e sua função é responder dúvidas e questionamentos relacionadas as instalações elétricas brasileiras com base no contexto, o qual pode incluir textos e/ou tabelas referentes as normas brasileiras (NBRs):
{context}
Question: {question}

"""
prompt = ChatPromptTemplate.from_template(template)

# Função para imprimir o contexto
def print_context(context):
    print("Contexto fornecido:", context)
    return context

# RAG pipeline com etapa intermediária para capturar e imprimir o contexto
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | RunnableMap({"context": print_context, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
query = "O que diz o tópico 10.2.9.1 da Norma NR 10?"
to_markdown(chain.invoke(query))

In [ ]:
query = "Qual seu nome?"
to_markdown(chain.invoke(query))

In [ ]:
query = "Quais são os requisitos de qualificação profissional para os trabalhadores que atuam em atividades abrangidas pela NR 10?"
to_markdown(chain.invoke(query))

In [ ]:
query = "Quais são as responsabilidades dos empregadores e dos trabalhadores em relação à NR 10?"
to_markdown(chain.invoke(query))

In [ ]:
query = "Que tipos de Equipamentos de Proteção Individual (EPIs) são exigidos pela NR 10 para os trabalhadores que atuam com eletricidade?"
to_markdown(chain.invoke(query))

In [ ]:
query = "Quais são as exigências da NR 10 para as instalações elétricas em áreas classificadas?"
to_markdown(chain.invoke(query))